### 案例需求：

在对一串数字进行排序时，把出现在某个组群中的数字放在其他数据之前。这种需求在绘制个性化用户界面时会遇到，我们可以用这种办法把重要消息或意外的事件优先显示在其它内容前面。

### 解决方案

sort函数key参数接受辅助函数。辅助函数的返回值用来确定列表中各元素的顺序。辅助函数用来判断元素是否在组群中，并根据判断结果添加相应的排序关键字（sort key）(这个可能与下面的第3个原因对应--修改比较元组)

In [8]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 5, 3, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


### 该方案使用到的重要Python特性 

1. 闭包（closure):闭包是指定义在某个作用域中的函数，这种函数引用(使用了)那个作用域中的变量。本例中，helper函数访问了sort_priority的group参数，这就是闭包。（只读）
2. Python的函数是一级对象（first-class object),函数可以像对象一样进行引用，赋值给变量，作为参数进行传递，并通过表达式及if语句对其进行比较和判断等等。本例中，helper函数作为参数传给了sort。
3. Python的元组比较规则：按照下标顺序进行依次比较

### 添加需求--返回出现优先级元件提示 

当用户界面出现了优先级高的元件前，我们需要提示用户是否应用这种优先级。此时我们需要换回一个值左侧相应的处理。

*直觉上的做法*

In [9]:
def sort_priority2(numbers,group):
    found = False
    def helper(x):
        if x in group:
            found = True # Seems simple
            return(0,  x)
        return (1, x)
    numbers.sort(key=helper)
    return found
            
found = sort_priority2(numbers, group)
print('Found:',found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


结果显示排序正确，但是found不对。
**原因：**
Python解释器解析引用（变量）时遍历作用域的顺序：
1. 当前函数作用域  -- helper
2. 任何外围作用域（例如：包涵当前函数的其他函数） -- sort_priority2
3. 包涵当前代码的模块作用域（也叫全局作用域，global scope) --- .py
4. 内置作用域（也就是包括len及str等函数的那个作用域)。Python 库环境

如果还没找到就抛出NameError异常。

赋值规则--闭包无法修改外围变量：

如果当前函数域（helper）已经定义了这个变量，就直接修改该变量；若当前作用域(helper)中没有这个变量，Python会在该作用域内重新定义同名的变量进行修改。这导致了helper无法**真实的**修改外围域中的变量。

这种设计的目的：防止函数中的局部变量修改污染函数外面的模块。这种污染可能导致一些难以检查的bug

### 获取闭包内的数据（强制修改外部局部变量）

**nonlocal 语句**

In [11]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return(1,x)
    numbers.sort(key=helper)
    return found

found = sort_priority3(numbers, group)
print('Found:',found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


nonlocal语句指明了修改的其实是闭包外的那个作用域（sort_priority3）的变量。这与global语句互为补充，global会直接修改稿模块作用域的变量。

**注意：** 尽量只在及其简单的函数里面使用这种nonlocal机制。滥用nonlocal会导致难以最终的bug。尤其是nonlocal语句与变量修改操作相关比较远时，还会导致代码难以理解。

如果这种需求比较复杂，那就应该将相关状态分装成辅助类（helper class）。下面的类将替代nonlocal的功能。它虽然有点长，但理解起来相当容易（其中有个名叫__call__的特殊方法，参见第23条）。 该类对应 helper函数。helper本身是一级类。

In [12]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

### Python2 方案

Python2 不支持nonlocal关键字。Python2 使用了不太优雅的Python作用域规则来解决。但这已经成了一种Python变成的习惯。

In [13]:
# Python2
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True
            return (0,x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]

print(sort_priority(numbers,group))

True


原理：Python在上级作用域中找到found，利用found列表类型的**mutable**（可变性），获取found列表后可以在闭包内通过found[0]=True，修改found的状态。

上级作用域中具有该特性的变量类型还有字典（dictionary）、集合或某个类的实例。

### 要点

1. 闭包可以引用上一级作用域中的变量。
2. 默认情况下闭包内无法有效的修改上一级作用域的变量
3. Python3 可以使用nonlocal语句强制修改上一级的变量
4. Python2 中使用可可变值（例如，包涵当元素的列表）实现nonlocal语句的功能。
5. 防止滥用 nonlocal语句，逻辑复杂时，考虑使用辅助类
6. mutable（可变性）的弊端值得研究